Machine learning prototyping notebook. Data preprocessing has already been tested and implemented in data_preproc.pu (samples/). 

In [5]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

import plotly.express as px
import matplotlib.pyplot as plt
import yfinance as yf

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeRegressor
from sklearn.multioutput import RegressorChain
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR, SVC
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, PredictionErrorDisplay, accuracy_score

Decision tree regressorGet the data, split into exo/endo and perform train/test split. Get the data, split into exo/endo and perform train/test split.

In [1]:
import os
import sys
import pandas as pd
import numpy as np

# IMPORT FUNCTIONS
sys.path.insert(0, '../sample')
import data_preproc

# LOAD FINANCIAL RATIOS AND ASSET PRICES
test_merge = pd.read_excel('../jupyter-notebooks/test_manual.xlsx')
test_merge = test_merge.loc[:, test_merge.columns != 'Unnamed: 0']
test_assets = pd.read_excel('../jupyter-notebooks/asset_prices.xlsx',index_col='Date')

# PREPROCESS FINANCIAL RATIOS DATA, REPLACE STRINGS WITH FLOATS
ML_data = test_merge.map(data_preproc.convert_placeholder_text_to_num)

# ENSURE THE TWO DATAFRAMES CONTAINING FINANCIAL RATIOS (ML_DATA) AND RETURNS (TEST_ASSETS) HAVE THE SAME ASSETS/TICKERS
ML_final = data_preproc.filter_ratios_returns(ML_data,test_assets)
# print(ML_final.head())

# RESAMPLE THE RETURNS FROM MONTHLY TO QUARTERLY, THEN BFILL AND FFILL
asset_prices = test_assets # MAKE A COPY
asset_prices.index = pd.to_datetime(asset_prices.index)
asset_prices = asset_prices.resample('Q').last()
asset_prices = asset_prices.bfill(axis=1)
asset_prices = asset_prices.ffill(axis=1)


# 
test = data_preproc.FRatioMLdata(ML_final,asset_prices,sector=None,returns_lead_by=-1)
#test.transform()
#print(test.train.head())

In [2]:
test = test.transform()

In [3]:
test

,EV,FCF,EBITDA,Revenue,ROE,Gross-Profit-Margin,Quick-Ratio,Debt / Equity,Returns
0,-0.360803,3.447233,6.393835,2.463991,-0.617978,-0.194444,0.000000,0.000000,0.041667
1,-0.251257,-0.184708,-0.630011,0.362394,-0.410596,0.862069,0.000000,0.023301,-0.153732
2,-0.026975,-0.004681,-0.004744,-0.004536,0.480392,0.000000,0.000000,0.000000,0.155340
3,0.258930,-2.478155,4.193577,-0.806291,1.000000,-0.309524,-0.600000,-0.427141,-0.036178
4,-0.475836,-0.002556,-0.002633,-0.002629,-1.864407,0.000000,0.000000,0.000000,0.148515
...,...,...,...,...,...,...,...,...,...
7,0.049659,-1.467892,-0.094140,-0.572862,0.232558,0.146119,0.000000,-0.341260,0.185185
8,-0.026540,3.187525,0.038230,0.513707,0.653846,0.531469,0.000000,0.034440,0.000000
9,-0.029439,-0.762979,0.221575,0.009796,0.000000,-0.089172,0.000000,-0.117264,-0.058140
10,-0.051483,-4.752607,0.241513,0.375513,-0.037037,0.154412,0.100000,0.075935,0.011765


In [6]:
X = test.iloc[:,:-1]
y = test.iloc[:,-1]

# scale the data
data_scaler_x = StandardScaler()
data_scaler_y = StandardScaler()

X = data_scaler_x.fit_transform(X.values)
y = data_scaler_y.fit_transform(y.values.reshape(-1,1))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 0)

In [7]:
X_train

array([[-8.14284749e-03,  5.94304077e-02,  4.28607164e-02, ...,
        -3.95243588e-02, -7.87666637e-02, -7.99015369e-02],
       [-2.33310479e-02,  5.91915304e-02,  4.26949220e-02, ...,
        -3.95243588e-02, -7.87666637e-02, -7.99015369e-02],
       [-2.98506515e-03,  2.11988257e-02,  5.07952837e-02, ...,
        -5.25994714e-02, -7.87666637e-02, -1.16733171e-01],
       ...,
       [-9.84651613e-03,  7.97416495e-02,  5.26134191e-02, ...,
        -2.78148163e-01, -2.21472135e-01, -8.10246311e-02],
       [-6.86912730e-02,  8.78735761e-01, -8.12688120e-01, ...,
        -4.69224555e-04, -1.55216023e-01, -9.64326945e-02],
       [-3.94816733e-02,  3.41951662e-01,  2.06389351e-01, ...,
         3.59369378e-02, -1.73903645e-01,  2.75425081e-03]])